In [ ]:
# Basic Sumamry Script

In [249]:
import pandas as pd

def count_unique_items_per_columns(file_path, column_names):

    df = pd.read_excel(file_path)
    
    for column_name in column_names:
       
        if column_name not in df.columns:
            print(f"Column '{column_name}' not found in the Excel file.")
        else:
            
            unique_counts = df[column_name].value_counts()
            
            print(f"\nUnique items and their frequencies in '{column_name}':")
            print(unique_counts)

file_path = 'updated_data.xlsx'
column_names = ['Sex', 'Eng_Prof', 'Race', 'Parental_Involvement_Score']
count_unique_items_per_columns(file_path, column_names)



Unique items and their frequencies in 'Sex':
Sex
1    96
2    81
Name: count, dtype: int64

Unique items and their frequencies in 'Eng_Prof':
Eng_Prof
7    131
3     21
2     14
4      6
1      3
5      1
6      1
Name: count, dtype: int64

Unique items and their frequencies in 'Race':
Race
1    84
2    48
3    44
4     1
Name: count, dtype: int64

Unique items and their frequencies in 'Parental_Involvement_Score':
Parental_Involvement_Score
14    21
17    18
15    17
16    14
13    14
19    13
20    11
12    11
10    10
11     9
9      7
21     6
8      5
6      5
7      4
18     4
23     3
24     3
22     2
Name: count, dtype: int64


In [ ]:
#Script for the first round of Cleaning

In [21]:
import pandas as pd

def clean_excel(file_path, output_file):
    df = pd.read_excel(file_path)


    df = df[(df['Q3_1'].isin(['A', 'B'])) & (df['Q4_1'].isin(['A', 'B']))]

    df = df.dropna(subset=['Q6_3', 'Q6_4', 'Q6_5', 'Q6_6', 'Q2_5'])

    df = df[~((df['Q3_1'] == 'A') & (df[['Q3_2', 'Q3_4']].isna().any(axis=1)))]

    df = df[~((df['Q4_1'] == 'A') & (df[['Q4_2', 'Q4_4']].isna().any(axis=1)))]

    df.loc[(df['Q3_1'] == 'B') & (df['Q3_2'].isna()), 'Q3_2'] = 'NA'
    df.loc[(df['Q3_1'] == 'B') & (df['Q3_4'].isna()), 'Q3_4'] = 'NA'

    df.loc[(df['Q4_1'] == 'B') & (df['Q4_2'].isna()), 'Q4_2'] = 'NA'
    df.loc[(df['Q4_1'] == 'B') & (df['Q4_4'].isna()), 'Q4_4'] = 'NA'

    value_mapping = {
        'A': 22459, 'B': 30451, 'C': 38443, 'D': 46435, 'E': 54427,
        'F': 62419, 'G': 70411, 'H': 78403, 'I': 86395, 'J': 94387,
        'K': 102379, 'L': 110371, 'M': 110372
    }
    
    df['Q2_5'] = df['Q2_5'].map(value_mapping)

    df['Per_Person_Budget'] = df['Q2_5'] / df['Total Family Members']

    df.to_excel(output_file, index=False)
    print(f"Cleaned data has been saved to {output_file}")

input_file = 'SDB data raw - Important Variables.xlsx'
output_file = 'cleaned_data_saha_maclaryn.xlsx'
clean_excel(input_file, output_file)

Cleaned data has been saved to cleaned_data.xlsx


In [ ]:
# Script used for second round of cleaning (Renaming the variables basically)

In [155]:
import pandas as pd

def update_values(file_path, output_file):
    
    df = pd.read_excel(file_path)

    df['Q3_1'] = df['Q3_1'].replace({'A': 1, 'B': 0})
    df['Q4_1'] = df['Q4_1'].replace({'A': 1, 'B': 0})

    replace_q6_3_4 = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
    df['Q6_3'] = df['Q6_3'].replace(replace_q6_3_4)
    df['Q6_4'] = df['Q6_4'].replace(replace_q6_3_4)

    replace_q6_5_6 = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}
    df['Q6_5'] = df['Q6_5'].replace(replace_q6_5_6)
    df['Q6_6'] = df['Q6_6'].replace(replace_q6_5_6)
    
    replace_q6_7_8 = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    df['Q6_7'] = df['Q6_7'].replace(replace_q6_7_8)
    df['Q6_8'] = df['Q6_8'].replace(replace_q6_7_8)

    df.to_excel(output_file, index=False)
    print(f"Updated data has been saved to {output_file}")

input_file = 'updated_data.xlsx'
output_file = 'updated_data1.xlsx'
update_values(input_file, output_file)


Updated data has been saved to updated_data1.xlsx


In [ ]:
#Just to check if Math scores increased with increasing Reading Test Score

In [207]:
import pandas as pd
import statsmodels.api as sm

df = pd.read_excel('updated_data.xlsx')

X = df['Average ReadRIT_18_19']
y = df['Average MathRIT_18_19']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())


                              OLS Regression Results                             
Dep. Variable:     Average MathRIT_18_19   R-squared:                       0.296
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     75.50
Date:                   Mon, 11 Nov 2024   Prob (F-statistic):           2.20e-15
Time:                           01:16:16   Log-Likelihood:                -881.58
No. Observations:                    182   AIC:                             1767.
Df Residuals:                        180   BIC:                             1774.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------


In [ ]:
#Script for the Linear Regression Model

In [237]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv('updated_data.csv')

X1 = df[['Sex', 'Race', 'Eng_Prof', 'Per_Person_Budget', 'Q6_3', 'Q6_4', 'Q6_5','Q6_6', 'Q6_7', 'Q6_8']]

y1 = df['Average ReadRIT_18_19']  

encoder = OneHotEncoder(drop='first')

encoded_X1 = encoder.fit_transform(X1[['Sex', 'Race']]).toarray()

encoded_columns = encoder.get_feature_names_out(['Sex', 'Race'])

encoded_X1_df = pd.DataFrame(encoded_X1, columns=encoded_columns)

X1 = pd.concat([encoded_X1_df, X1[['Eng_Prof','Per_Person_Budget', 'Q6_3', 'Q6_4', 'Q6_5','Q6_6', 'Q6_7', 'Q6_8']]], axis=1)

X1 = sm.add_constant(X1)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.1, random_state=42)

model1 = sm.OLS(y1_train, X1_train).fit()

print("Model 1 Summary: ")
print(model1.summary())

y1_pred = model1.predict(X1_test)

print(f"Model 1 R-squared on test data: {model1.rsquared}")

Model 1 Summary: 
                              OLS Regression Results                             
Dep. Variable:     Average ReadRIT_18_19   R-squared:                       0.276
Model:                               OLS   Adj. R-squared:                  0.216
Method:                    Least Squares   F-statistic:                     4.629
Date:                   Mon, 11 Nov 2024   Prob (F-statistic):           2.43e-06
Time:                           01:28:15   Log-Likelihood:                -818.36
No. Observations:                    159   AIC:                             1663.
Df Residuals:                        146   BIC:                             1703.
Df Model:                             12                                         
Covariance Type:               nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------